In [1]:
import csv
import ast
import sys
import operator
!conda install --yes --prefix {sys.prefix} pandas
import pandas as pd

lexicon_file = "lexicon/NRC-Emotion-Lexicon-v0.92/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"
lexicon_df = pd.read_csv(lexicon_file,  names=["word", "emotion", "association"], sep='\t')
methods = ["Topic Rank","Single Rank","Position Rank","Multipartite Rank"]

def method_formatter(method):
    """
    Formats the method's name for writing
    :param method: method's name
    :return: formatted name
    """
    return method.lower().replace(' ','_')
    
for index,method in enumerate(methods):
    with open('results/keywords.csv', 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        
        with open('results/'+method_formatter(method)+'_sentiments.csv', 'w', newline="") as csvfile2:
            filewriter = csv.writer(
                csvfile2,
                delimiter=',',
                quotechar='|',
                quoting=csv.QUOTE_MINIMAL
            )
            
            filewriter.writerow(
                [ 
                    'Rank','Song','Artist','Year','Main Emotion',"anger","anticipation","disgust","fear","joy","sadness","surprise","trust","neutral","na"
                ]
            )
            
            for song in reader:
                song_rank = song[0]
                song_name = song[1]
                song_artist = song[2]
                song_year = song[3]
                song_keywords = song[3+index+1]
    
                emotion_score = {
                    "anger":0,
                    "anticipation":0,
                    "disgust":0,
                    "fear":0,
                    "joy":0,
                    "sadness":0,
                    "surprise":0,
                    "trust":0,
                    "neutral":0,
                    "na":0,
                }
    
                for word in song_keywords.split(';'):
                    clean_word = word.replace("'",'')
                    matches = lexicon_df.loc[lexicon_df['word'] == clean_word]
                    if not matches.empty:
                        for emotions in matches.iterrows():
                            is_neutral = True
                            for emotion in emotions:
                                if type(emotion) != int \
                                        and emotion.emotion !='positive'\
                                        and emotion.emotion !='negative':
                                    if emotion.association == 1:
                                        is_neutral = False
                                    emotion_score[emotion.emotion] += emotion.association
                            if is_neutral:
                                emotion_score["neutral"] = 1
    
                total_score = 0
                for emotion in emotion_score:
                    total_score += emotion_score[emotion]
    
                for emotion in emotion_score:
                    if total_score != 0:
                        emotion_score[emotion] = emotion_score[emotion]*100/total_score
                    else:
                        emotion_score['na'] = 100
    
                main_emotion = max(emotion_score.items(), key=operator.itemgetter(1))[0]
                
                filewriter.writerow(
                    [
                        song_rank,song_name,song_artist,song_year,main_emotion,emotion_score["anger"],
                        emotion_score["anticipation"], emotion_score["disgust"], emotion_score["fear"],
                        emotion_score["joy"], emotion_score["sadness"], emotion_score["surprise"],
                        emotion_score["trust"],emotion_score["neutral"], emotion_score["na"]
                    ]
                )
                
            print(method_formatter(method)+".csv is done")

Solving environment: ...working... done

# All requested packages already installed.

topic_rank.csv is done
single_rank.csv is done
position_rank.csv is done
multipartite_rank.csv is done
